In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.linear_model import LinearRegression

np.random.seed(42)
RSTATE = 42

# Supervised Learning — Linear Regression

## 1. What is Regression?

Regression is a fundamental task in **supervised learning**. Its goal is to **predict a continuous numerical value**.

Unlike *classification*, which predicts a category (e.g., “dog” or “cat”, `0` or `1`), regression predicts a **numeric value** (e.g., 150,000 CZK, 25.5 °C).

We aim to find a **mathematical function (model)** that best describes the relationship between **input features** and a **target variable**.

* **Example:**
    * **Features (X):** `apartment size`, `number of rooms`, `distance from city center`
    * **Target (y):** `apartment price`

In this exercise, we focus on **linear regression**. It assumes that the relationship is linear — meaning it can be described by a **straight line** (in one dimension) or a **hyperplane** (in multiple dimensions).

The goal is to find such parameters (weights) of this line so that it is “as close as possible” to all training data points. This process is called training the model.


## Mathematical Background

### Linear regression

The model attempts to fit a straight line (or a hyperplane in higher dimensions) through the data.

Prediction (hypothesis):
$$
\hat{y}_i = w^\top x_i + b
$$
where \( w \) are the weights (slope of the line) and \( b \) is the bias (y-intercept).

The loss function we minimize is the **Mean Squared Error (MSE)**:
$$
\mathcal{L}_{\mathrm{MSE}}(w,b) = \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2
$$

This function simply says: “Sum the squared distances between each prediction \( \hat{y}_i \) and the actual value \( y_i \), and divide by the number of samples.”

---

### Gradient Descent

Our goal is to find parameters \( w \) and \( b \) that minimize \( \mathcal{L}_{\mathrm{MSE}} \). We do this using **gradient descent**.

It is an iterative process where we adjust \( w \) and \( b \) “step by step” in the direction that reduces the error the most. This direction is given by the negative gradient (partial derivatives).

Gradients for **basic MSE**:
$$
\nabla_w \mathcal{L} = \frac{2}{n} X^\top (Xw + b\mathbf{1} - y)
$$
$$
\frac{\partial \mathcal{L}}{\partial b} = \frac{2}{n} \sum_{i=1}^n (\hat{y}_i - y_i)
$$

Parameter updates are performed repeatedly as:
$$
w \leftarrow w - \eta \nabla_w \mathcal{L}
$$
$$
b \leftarrow b - \eta \frac{\partial \mathcal{L}}{\partial b}
$$

where \( \eta \) (eta) is the **learning rate**, which controls the step size taken in each iteration.

---

### Improvement: Why Regularization? (Preventing Overfitting)

When we have many features, or when some features are highly correlated (multicollinearity), the model may **overfit**.

This means it fits the training data extremely well but assigns **very large weights** to some features. As a result, the model becomes sensitive to small changes in the input and **fails to generalize** to new, unseen data (e.g., validation or test sets).

**L2 regularization (Ridge regression)** helps prevent this by adding a penalty term that discourages large weights.

Loss function for **Ridge Regression** (MSE + L2 penalty):
$$
\mathcal{L}_{\mathrm{Ridge}} = \mathcal{L}_{\mathrm{MSE}} + \alpha \|w\|_2^2
= \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2 + \alpha \|w\|_2^2
$$

* \( \alpha \) (alpha) is a hyperparameter that controls the strength of regularization
* \( \|w\|_2^2 \) is the squared L2 norm of the weight vector (sum of squared weights)

The gradient changes only for \( w \) (bias \( b \) is typically not regularized):

$$
\nabla_w \mathcal{L}_{\mathrm{Ridge}}
= \underbrace{\frac{2}{n} X^\top (Xw + b\mathbf{1} - y)}_{\text{original gradient}}
+ \underbrace{2 \alpha w}_{\text{regularization term}}
$$

Thus, at each gradient-descent step, we both decrease the MSE and “pull” the weights toward zero (known as *weight decay*).

## Optimization Methods: Batch vs. Stochastic vs. Mini-Batch GD

Our `LinearRegressorGD` uses **Batch Gradient Descent**, because it is the easiest method to understand conceptually. However, it is not the only method — and in practice, not even the most common one.

There are three main strategies for computing gradients and updating model weights:

### 1. Batch Gradient Descent (what we use)
In each epoch, the gradient (the direction of steepest error decrease) is computed from the **entire training dataset** at once:
$$
\nabla_w \mathcal{L} = \frac{2}{n} \sum_{i=1}^n (X_i^\top (\hat{y}_i - y_i)) + 2 \alpha w
$$

Then we make **a single update** to the weights:

\[
w \leftarrow w - \eta \nabla_w \mathcal{L}
\]

* **Advantages:**
  * Very stable and smooth convergence.
  * Guarantees finding the global minimum (for convex functions like MSE).

* **Disadvantages:**
  * **Extremely slow** for large datasets (e.g., 1M samples) — it must process *all* data before taking one step.
  * High memory requirements (entire dataset must fit into RAM).

---

### 2. Stochastic Gradient Descent (SGD)
In each epoch, the training data are shuffled. Then we process **one sample at a time**, computing the gradient and updating weights **after each individual sample**.

If the dataset has \(n\) samples, we perform \(n\) small weight updates per epoch:

$$
\nabla_w \mathcal{L}_i = 2 (X_i^\top (\hat{y}_i - y_i)) + 2 \alpha w \quad \text{(gradient for a single sample } i)
$$

\[
w \leftarrow w - \eta \nabla_w \mathcal{L}_i
\]

* **Advantages:**
  * **Very fast** iterations — the model starts learning immediately.
  * Low memory usage (one sample at a time).
  * Noise in gradients (from individual samples) can help escape shallow local minima.

* **Disadvantages:**
  * Convergence is noisy and unstable; weights oscillate around the optimum.
  * Usually does not fully converge — often requires gradually decreasing the learning rate \((\eta)\).

---

### 3. Mini-Batch Gradient Descent (the “golden mean”)
This is a compromise and the **most widely used method in practice** (especially in deep learning).

The dataset is split into small "batches" (**mini-batches**) of size \(k\) (e.g., \(k = 32\), 64, or 256 samples). Gradients are computed and weights updated after each batch.

* **Advantages:**
  * Much faster than Batch GD.
  * More stable convergence than SGD.
  * Fully benefits from vectorization and modern GPUs.

* **Disadvantages:**
  * Introduces another hyperparameter (`batch_size`).

---

**Conclusion:**  
In this notebook, we stick with **Batch GD** for clarity and stability. In the classification notebook, we will explore a method whose update pattern (sample-by-sample) is much closer to **SGD**.


In [ ]:
# Data Preparation

# 1. Generate synthetic data
# We use the 'make_regression' function from scikit-learn
# n_features=1: Only one feature (X), so we can easily visualize it
# noise=18.0: Amount of noise; the higher it is, the harder the task becomes
X_reg, y_reg = make_regression(n_samples=350, n_features=1, noise=18.0, random_state=RSTATE)

# 2. Split into Train (60%), Validation (20%), Test (20%)
# This is a key step for proper model evaluation.
# We train the model on 'Train', tune hyperparameters on 'Validation',
# and measure final performance on 'Test' (which the model has never seen).

# Step 1: Split into Train (60%) and "Temporary" (40%)
Xr_tr, Xr_tmp, yr_tr, yr_tmp = train_test_split(X_reg, y_reg, test_size=0.4, random_state=RSTATE)

# Step 2: Split "Temporary" (40%) into Validation (20%) and Test (20%)
# (test_size=0.5 means 50% of 40%, which equals 20% of the full dataset)
Xr_va, Xr_te, yr_va, yr_te = train_test_split(Xr_tmp, yr_tmp, test_size=0.5, random_state=RSTATE)

print("--- Dataset shapes (before scaling) ---")
print(f"Train:      {Xr_tr.shape}, {yr_tr.shape}")
print(f"Validation: {Xr_va.shape}, {yr_va.shape}")
print(f"Test:       {Xr_te.shape}, {yr_te.shape}")
print("-" * 30)


# 3. Standardization (Scaling) of data
# Why? Gradient Descent (GD) and L2 regularization work much better
# and faster when all features have a similar scale.
# Standardization transforms data to have mean 0 and standard deviation 1.

# IMPORTANT: Preventing Data Leakage
# The scaler (StandardScaler) must be 'fit' (learn mean & std)
# ONLY on training data (Xr_tr).
sc_r = StandardScaler().fit(Xr_tr)

# The trained scaler is then ONLY APPLIED to validation and test data
Xr_tr_s = sc_r.transform(Xr_tr)
Xr_va_s = sc_r.transform(Xr_va)
Xr_te_s = sc_r.transform(Xr_te)

# The target variable (y) usually does not need to be scaled for linear regression.

print("--- Scaling check (mean / std dev) ---")
print(f"Train (scaled):      {Xr_tr_s.mean():.2f} / {Xr_tr_s.std():.2f}")
print(f"Validation (scaled): {Xr_va_s.mean():.2f} / {Xr_va_s.std():.2f}")
print(f"Test (scaled):       {Xr_te_s.mean():.2f} / {Xr_te_s.std():.2f}")


In [ ]:
# Let's take a look at the data we generated.
# We will plot the TRAINING set (before scaling, so the axes remain readable).

plt.figure(figsize=(10, 6))
plt.scatter(Xr_tr, yr_tr, alpha=0.7, label="Training data")
plt.xlabel("Feature X")
plt.ylabel("Target value y")
plt.title("Generated regression data (Train set)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
class LinearRegressorGD:
    """
    Implementation of linear regression.

    Trained using Batch Gradient Descent (GD).
    Includes L2 (Ridge) regularization and Early Stopping.
    """

    def __init__(self, lr: float = 0.05, n_epochs: int = 1500,
                 fit_intercept: bool = True, alpha: float = 0.0,
                 early_stopping_patience: int = 20):
        """
        Initialize model hyperparameters.

        lr: Learning rate
        n_epochs: Maximum number of epochs (passes over data)
        fit_intercept: Whether to learn 'b' (bias/intercept)
        alpha: Strength of L2 regularization (0 = no regularization)
        early_stopping_patience: Number of epochs without improvement on
                                 the validation set before training stops.
        """
        self.lr = lr
        self.n_epochs = n_epochs
        self.fit_intercept = fit_intercept
        self.alpha = alpha  # L2 regularization strength
        self.early_stopping_patience = early_stopping_patience

        # Internal variables for learned parameters and history
        self.w = None
        self.b = 0.0
        self.train_rmse_history = []
        self.val_rmse_history = []

    def fit(self, X: np.ndarray, y: np.ndarray,
            X_val: np.ndarray = None, y_val: np.ndarray = None):
        """
        Train the model on (X, y), optionally using (X_val, y_val)
        for early stopping.
        """
        n, d = X.shape  # n = number of samples, d = number of features
        self.w = np.zeros(d, dtype=float)
        self.b = 0.0

        # --- Early Stopping setup ---
        use_es = X_val is not None and y_val is not None
        best_val_rmse = np.inf
        patience_counter = 0
        best_w = self.w.copy()
        best_b = self.b
        # ----------------------------

        print(f"Training starts (max {self.n_epochs} epochs, L2 alpha={self.alpha})...")

        for epoch in range(self.n_epochs):
            # 1. Predictions (forward pass)
            y_hat = X @ self.w + (self.b if self.fit_intercept else 0.0)

            # 2. Compute error (residuals)
            residual = (y_hat - y)

            # 3. Compute gradients (Batch GD — from all samples at once)

            # Gradient for weights 'w' with L2 regularization
            # (2.0 / n) * (X.T @ residual)  <- gradient from MSE
            # + 2.0 * self.alpha * self.w   <- gradient from L2 penalty
            grad_w = (2.0 / n) * (X.T @ residual) + 2.0 * self.alpha * self.w

            grad_b = 0.0
            if self.fit_intercept:
                # Gradient for bias 'b' (bias is typically not regularized)
                grad_b = (2.0 / n) * np.sum(residual)

            # 4. Weight update (step against the gradient direction)
            self.w -= self.lr * grad_w
            if self.fit_intercept:
                self.b -= self.lr * grad_b

            # --- Log history and check Early Stopping ---
            train_rmse = np.sqrt(mean_squared_error(y, y_hat))
            self.train_rmse_history.append(train_rmse)

            if use_es:
                val_rmse = np.sqrt(mean_squared_error(y_val, self.predict(X_val)))
                self.val_rmse_history.append(val_rmse)

                # Save the best model according to validation error
                if val_rmse < best_val_rmse - 1e-5:  # Small tolerance for improvement
                    best_val_rmse = val_rmse
                    best_w = self.w.copy()
                    best_b = self.b
                    patience_counter = 0
                else:
                    patience_counter += 1

                # If the model has not improved for 'patience' epochs, stop training
                if patience_counter >= self.early_stopping_patience:
                    print(f"INFO: Early stopping at epoch {epoch + 1} (Val RMSE: {best_val_rmse:.4f})")
                    break

            # Print progress every N epochs
            if (epoch + 1) % (self.n_epochs // 10) == 0 and not use_es:
                print(f"  Epoch {epoch+1}/{self.n_epochs}, Train RMSE: {train_rmse:.4f}")

        # --- End of training ---
        if use_es:
            # Restore the best weights found on the validation set
            print(f"INFO: Training finished. Restoring model from epoch {epoch + 1 - patience_counter}.")
            self.w = best_w
            self.b = best_b
        else:
            print("INFO: Training finished (reached max epochs).")

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Make predictions on new data X.
        """
        if self.w is None:
            raise RuntimeError("Model is not trained. Call 'fit' first.")

        return X @ self.w + (self.b if self.fit_intercept else 0.0)


In [ ]:
# --- Helper functions for advanced metrics ---
def adjusted_r2(r2, n, p):
    """
    Computes Adjusted R^2.
    Penalizes the model for adding features (p) that do not provide benefit.
    n = number of samples, p = number of features
    """
    return 1.0 - (1.0 - r2) * (n - 1) / (n - p - 1)

def mape(y_true, y_pred, eps=1e-8):
    """
    Mean Absolute Percentage Error (MAPE).
    Reports the average error in percent. It is sensitive to zero values in y_true.
    """
    mask = np.abs(y_true) > eps
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


# --- 1. Model initialization ---
# Set a high n_epochs (e.g., 5000); Early Stopping will likely halt earlier.
# alpha=0.01: Use mild L2 regularization.
# early_stopping_patience=50: Stop training if Val RMSE does not improve for 50 consecutive epochs.
reg = LinearRegressorGD(
    lr=0.05,
    n_epochs=5000,
    alpha=0.01,
    early_stopping_patience=50
)

# --- 2. Model training ---
# Train on scaled training data (Xr_tr_s, yr_tr)
# and use scaled validation data (Xr_va_s, yr_va) for Early Stopping.
reg.fit(Xr_tr_s, yr_tr, X_val=Xr_va_s, y_val=yr_va)

# --- 3. Predictions on all splits ---
# (Use the learned model on scaled data)
yr_pred_tr = reg.predict(Xr_tr_s)
yr_pred_va = reg.predict(Xr_va_s)
yr_pred_te = reg.predict(Xr_te_s)

# --- 4. Evaluation on the TEST set ---
# These are the final numbers we report for model performance.
print("\n" + "---" * 15)
print("Final performance on the TEST set:")
print("---" * 15)

n_test = yr_te.shape[0]
p_features = Xr_te_s.shape[1]

mse  = mean_squared_error(yr_te, yr_pred_te)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(yr_te, yr_pred_te)
r2   = r2_score(yr_te, yr_pred_te)
adjr2 = adjusted_r2(r2, n=n_test, p=p_features)
mape_v = mape(yr_te, yr_pred_te)

print(f"RMSE (Root Mean Squared Error): {rmse:.3f}")
print(f"MAE (Mean Absolute Error):      {mae:.3f}")
print(f"MAPE (Mean Absolute % Error):   {mape_v:.2f} %")
print(f"R² (R-squared):                 {r2:.3f}")
print(f"Adjusted R²:                    {adjr2:.3f}")

# It is also good practice to check the mean error (model bias)
me = float(np.mean(yr_pred_te - yr_te))
print(f"ME (Mean Error / Bias):         {me:.3f}")


## Explanation of Metrics (Model Diagnostics)

A single number (such as RMSE) never tells the whole story. It is important to look at multiple metrics, because each one measures a different aspect of model performance.

$$\phantom{}$$

* **RMSE (Root Mean Squared Error)**
    * **What it measures:** “What is the *typical* error of my model, in the same units as the target \(y\)?”
      (For example, if the target is in CZK, RMSE is also in CZK.)
    * **Interpretation:** Lower is better.
    * **Property:** Since the errors are *squared*, RMSE is **very sensitive to outliers**. A single very bad prediction increases RMSE much more than it would increase MAE.

* **MAE (Mean Absolute Error)**
    * **What it measures:** The *average* error of the model, in the same units as the target.
    * **Interpretation:** Lower is better.
    * **Property:** Uses absolute values, so it is **less sensitive to outliers** than RMSE.
    * **Tip:** If **RMSE is much larger than MAE**, your model likely has a few very large errors (probably due to outliers).
    *(diagram: RMSE vs MAE effect of outliers)*

* **MAPE (Mean Absolute Percentage Error)**
    * **What it measures:** “On average, by how many *percent* does the model miss the true value?”
    * **Interpretation:** Lower is better. Great metric for business communication (e.g., “Our price model has a 5% average error.”)
    * **Property:** Can be problematic when target values \(y\) are zero or close to zero (division by zero issues).

* **R² (R-squared / Coefficient of Determination)**
    * **What it measures:** What *percentage of variability* in \(y\) the model can explain.
    * **Interpretation:**
        * **1.0:** Perfect model
        * **0.7:** Model explains 70% of the variance
        * **0.0:** Model is no better than predicting the mean of \(y\)
        * *(can even be negative if the model is worse than always predicting the mean)*
    * **Property:** R² **never decreases** when you add features — even if they are useless.

* **Adjusted R²**
    * **What it measures:** A modified version of R² that **penalizes adding unnecessary features** that do not actually improve the model.
    * **Interpretation:** Always ≤ R². Much more reliable when comparing models with *different numbers of features*.

* **ME (Mean Error / Bias)**
    * **What it measures:** “Does my model systematically *overestimate* or *underestimate*?”
    * **Interpretation:** Unlike MAE, errors do not use absolute values, so positive and negative errors **cancel each other out**.
        * **ME > 0:** Model tends to overpredict
        * **ME < 0:** Model tends to underpredict
    * **Property:** We want this value to be as **close to zero** as possible. A non-zero ME indicates systematic bias in the model.


## Training History (RMSE vs. Epoch)

This plot is crucial for diagnosing the learning process itself. It shows how the model error changed with each epoch (pass through the data).

* **Train RMSE (blue line):** Shows how well the model fits the data it is trained on. We expect this curve to generally decrease.
* **Validation RMSE (orange line):** Shows how well the model **generalizes** to unseen data. This is the most important curve for model evaluation.

**What we look for in the plot:**

* **Ideal scenario:** Both curves decrease and stabilize at a low level close to each other.
* **Overfitting:** The blue (Train) curve drops low, but the orange (Validation) curve starts rising again at some point.  
  The model “memorizes” the training data but loses its ability to generalize.
* **Underfitting:** Both curves stay high. The model is too simple and fails to capture even the underlying trend in the training data.
* **Early Stopping marker (red line):** Indicates the epoch where training stopped.  
  This is the point with the lowest validation error. Without early stopping, the model would begin to overfit (the orange curve would gradually rise).


In [ ]:
# Retrieve training error history from the trained model
train_history = reg.train_rmse_history
val_history = reg.val_rmse_history

# Find the epoch where the validation RMSE was the lowest
best_epoch = np.argmin(val_history)
best_val_rmse = val_history[best_epoch]

plt.figure(figsize=(12, 7))
plt.plot(train_history, label="Train RMSE", lw=2)
plt.plot(val_history, label="Validation RMSE", lw=2)

# Mark the best epoch from which the model was restored
plt.axvline(best_epoch, color='red', linestyle='--',
            label=f"Best model (Epoch: {best_epoch}, RMSE: {best_val_rmse:.3f})")

plt.xlabel("Epoch")
plt.ylabel("RMSE (Root Mean Squared Error)")
plt.title("Training History (Early Stopping)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Trim the y-axis to highlight differences (ignore the first ~5–10 epochs)
plt.ylim(bottom=0, top=np.max(val_history[5:]) * 1.1)
plt.show()


## Final Model vs. Test Data

This plot shows the final outcome of our work.

* **Blue points:** These are the **test data** (`Xr_te_s`, `yr_te`). They are the real (“ground truth”) values that the model **never saw** during training or validation.
* **Red line:** This is our trained model — its **predictions**. It shows what output \(y\) the model would predict for any value of \(x\).
* **Light-red band:** This is a heuristic band of \(\pm 2 \times \text{RMSE}\). If the errors are normally distributed, we would expect roughly **95% of all test points** to lie within this band.

**What we look for:**
We want the red line to cut through the cloud of blue points as well as possible, and for the points to be as close to this line as possible.


In [ ]:
# Regression line vs. Test data

# Create an X-axis for a smooth line
# (we must use the standardized space in which the model was trained)
min_x_s = Xr_te_s.min()
max_x_s = Xr_te_s.max()
# Extend the range by 10% for a nicer plot
padding = (max_x_s - min_x_s) * 0.1
xs_std = np.linspace(min_x_s - padding, max_x_s + padding, 200).reshape(-1, 1)

# Compute predictions for this line
ys_pred_line = reg.predict(xs_std)

# Reload RMSE from cell 9 for plotting the band
# rmse = np.sqrt(mean_squared_error(yr_te, yr_pred_te))

plt.figure(figsize=(12, 7))

# 1. Plot test data (ground truth)
plt.scatter(Xr_te_s, yr_te, alpha=0.7, label="Test data (ground truth)")

# 2. Plot our learned line
plt.plot(xs_std, ys_pred_line, color='red', lw=3, label="Regression line (prediction)")

# 3. (Optional) Plot the heuristic ±2*RMSE band
# This band should contain ~95% of all points
plt.fill_between(xs_std.ravel(), ys_pred_line - 2*rmse, ys_pred_line + 2*rmse,
                 color='red', alpha=0.1, label=f"Heuristic band (±2×RMSE ≈ ±{2*rmse:.2f})")

plt.xlabel("X (standardized)")
plt.ylabel("y (target value)")
plt.title("Regression: Final model vs. Test set")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


## Residual Analysis (Errors vs. Predictions)

This is the most important diagnostic plot for regression.  
A **residual** is simply another name for the model error (`actual − prediction`).

**What we want to see:**  
A “random cloud of points” centered around the zero line.

**What we do NOT want to see:** Any visible pattern.  
A pattern means there is still structure in the data that the model failed to capture. Examples of problematic patterns:

* **Trumpet / funnel shape (heteroscedasticity):**  
  The prediction error increases as predictions get larger.  
  The model is more certain for smaller values and worse for larger values.

* **Arc / parabola:**  
  The model systematically misses a nonlinear relationship.


In [ ]:
# Residual analysis (Errors vs. Predictions)

# Residuals = (actual value - predicted value)
resid = yr_te - yr_pred_te

plt.figure(figsize=(12, 7))

# X-axis: predicted value
# Y-axis: residual (error)
plt.scatter(yr_pred_te, resid, alpha=0.7, edgecolors='k', s=60)

# Add a zero line — residuals should be centered around it
plt.axhline(0.0, color='red', linestyle='--', lw=2)

plt.xlabel("Predicted value (y_hat)")
plt.ylabel("Residual (y - y_hat)")
plt.title("Residual Analysis (Test set)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

## QQ Plot of Residuals

This plot checks whether our errors (residuals) are **normally distributed**.  
This is an important assumption for many statistical tests (although it is not strictly required for training the model itself).

**What we want to see:**  
The points should lie as close as possible to the red line.

* If the points deviate at the ends (“tails”), it means we have more large errors (outliers) than would be expected under a normal distribution.


In [ ]:
# QQ Plot of residuals (Test set)

plt.figure(figsize=(8, 8))
# stats.probplot compares the distribution of our data (resid)
# with a theoretical normal distribution ("norm")
stats.probplot(resid, dist="norm", plot=plt)
plt.title("QQ Plot of Residuals (Normality Check)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


## K-Fold Cross-Validation

A simple split into `train`, `validation`, and `test` sets is useful, but it has one drawback: the result (e.g., RMSE on the validation set) heavily depends on which data *randomly* ended up in each set.  
What if we are unlucky and the validation set contains only “difficult” examples? Then our performance estimate would be unnecessarily pessimistic.

**K-Fold Cross-Validation** solves this problem neatly:

1. The entire training dataset is split into \(K\) (e.g., 5 or 10) equally sized parts (“folds”).
2. The process is repeated \(K\) times:
   * One fold becomes the **validation set**
   * The remaining \(K-1\) folds are used as the **training set**
   * The model is trained and evaluated
3. This results in \(K\) different performance estimates (e.g., \(K\) values of RMSE)
4. These values are averaged to obtain a much more stable and reliable estimate of model performance.

**Key rule (preventing data leakage):**  
The scaler (or any other data transformation) must **always be fit again inside each fold** using only the training data of that fold.


In [ ]:
# Implementation of 5-Fold Cross-Validation

from sklearn.model_selection import KFold

# For demonstrating CV, use the original unscaled data
# (combine training and validation sets; leave the test set aside)
X_cv_data = np.vstack((Xr_tr, Xr_va))
y_cv_data = np.concatenate((yr_tr, yr_va))

# Initialize the K-Fold splitter
kf = KFold(n_splits=5, shuffle=True, random_state=RSTATE)

cv_rmse_scores = []
fold_counter = 1

print("Starting 5-Fold Cross-Validation...")
# Loop over 5 folds
for tr_idx, va_idx in kf.split(X_cv_data):
    # 1. Split data into the current fold
    Xtr_fold, Xva_fold = X_cv_data[tr_idx], X_cv_data[va_idx]
    ytr_fold, yva_fold = y_cv_data[tr_idx], y_cv_data[va_idx]

    # 2. FIT THE SCALER ONLY ON THE TRAINING SPLIT OF THE FOLD
    sc_fold = StandardScaler().fit(Xtr_fold)

    # 3. Transform both splits using the fitted scaler
    Xtr_s_fold = sc_fold.transform(Xtr_fold)
    Xva_s_fold = sc_fold.transform(Xva_fold)

    # 4. Train the model (without Early Stopping, for a fixed number of epochs)
    #    Early Stopping doesn't make sense here because there is no separate validation set
    mdl_fold = LinearRegressorGD(lr=0.05, n_epochs=500, alpha=0.01)
    mdl_fold.fit(Xtr_s_fold, ytr_fold)  # Train without a validation split

    # 5. Evaluate on the validation fold
    y_pred_fold = mdl_fold.predict(Xva_s_fold)
    rmse_fold = np.sqrt(mean_squared_error(yva_fold, y_pred_fold))

    print(f"  Fold {fold_counter}/5 RMSE: {rmse_fold:.3f}")
    cv_rmse_scores.append(rmse_fold)
    fold_counter += 1

# --- Final CV result ---
print("---" * 10)
print("K-Fold Cross-Validation result (RMSE):")
print(f"  Mean RMSE: {np.mean(cv_rmse_scores):.3f}")
print(f"  Std. dev. of RMSE: {np.std(cv_rmse_scores):.3f}")

# This average result is a much more reliable performance estimate
# than RMSE from a single validation split.


## Models from Scikit-learn

Our `LinearRegressorGD` is great for understanding how things work “under the hood.”  
In practice, however, we almost always use optimized and robust implementations from the `scikit-learn` library.

The most common linear regression models in `sklearn`:

* **`LinearRegression`**
    * **What it is:** Standard linear regression.
    * **How it works:** Instead of Gradient Descent, it uses a closed-form analytical solution (the **Normal Equation**). Very fast for small and medium-sized datasets.
    * **Equivalent to:** Our model with `alpha=0` (but solved analytically, not iteratively).

* **`Ridge`**
    * **What it is:** Linear regression with **L2 regularization**.
    * **How it works:** Also uses an analytical solution, adjusted with the L2 penalty term. Preferred when you have many features or risk of multicollinearity.
    * **Equivalent to:** Our `LinearRegressorGD` with `alpha > 0`.

* **`Lasso`**
    * **What it is:** Linear regression with **L1 regularization**  
      \(\alpha \sum |w_i|\)
    * **How it works:** Unlike L2, L1 can “zero out” weights of irrelevant features.
      Excellent for **automatic feature selection**.
    * **Equivalent to:** Conceptually similar, but uses a different penalty (L1).

* **`ElasticNet`**
    * **What it is:** A combination of L1 and L2 regularization.  
      Takes the best of both Ridge and Lasso.

* **`SGDRegressor`**
    * **What it is:** Linear regression trained using **Stochastic Gradient Descent**.
    * **How it works:** Updates weights after each sample (or mini-batch).
    * **Equivalent to:** Exactly what we discussed in the “Batch vs. Stochastic GD” section.  
      Crucial for **very large (Big Data)** datasets that do not fit into memory, because it learns “online,” sample by sample.  
      Requires careful data scaling (which we already know how to do!).


## About the Diabetes Dataset

The Diabetes dataset is a classic regression dataset included in `scikit-learn`.  
It contains **medical diagnostic measurements** of patients and the goal is to predict a **quantitative disease progression score** one year after baseline.

**Dataset characteristics:**

| Property | Value |
|---|---|
Number of samples | 442 patients  
Number of features | 10 numeric features  
Task | Regression — predict disease progression  
Target | Disease progression after one year (quantitative score)  

**Features description (all continuous numeric variables):**

| Feature | Meaning |
|---|---|
`age` | Age of patient  
`sex` | Biological sex  
`bmi` | Body mass index  
`bp` | Average blood pressure  
`S1`–`S6` | Blood serum measurements (lipid & sugar profile)  

Note: in this dataset, all input features are **already standardized** to have zero mean and unit variance, but we still apply a `StandardScaler` inside a `Pipeline` for consistency and to demonstrate the correct workflow for general datasets.


In [ ]:
# Application to real-world data (Diabetes)

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# 1. Load data
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

print(f"'Diabetes' dataset loaded:")
print(f"  Number of features: {X.shape[1]}")
print(f"  Number of samples:  {X.shape[0]}")

# 2. Train–test split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=RSTATE)

# 3. Build a Pipeline
# A Pipeline is a "container" that executes steps sequentially:
# 1) 'scale': Standardizes the data via StandardScaler
# 2) 'model': Trains a Ridge regression
# This automatically prevents data leakage (the scaler is fit only on X_tr).
pipe_ridge = Pipeline([
    ('scale', StandardScaler()),
    ('model', Ridge(alpha=1.0))  # alpha=1.0 is the default regularization strength
])

# 4. Training
print("\nTraining Pipeline (StandardScaler + Ridge)...")
pipe_ridge.fit(X_tr, y_tr)
print("Training finished.")

# 5. Evaluation on the test set
y_pred = pipe_ridge.predict(X_te)

rmse_sk = np.sqrt(mean_squared_error(y_te, y_pred))
r2_sk = r2_score(y_te, y_pred)

print("\n--- Performance on the Diabetes Test Set ---")
print(f"RMSE: {rmse_sk:.3f}")
print(f"R²:   {r2_sk:.3f}")

# Show learned coefficients (weights)
coefs = pipe_ridge.named_steps['model'].coef_
print(f"\nLearned coefficients (weights): \n{coefs.round(2)}")


### Commentary on Results

The model trained on the Diabetes dataset using a standardized **Ridge regression** achieved:

- **RMSE ≈ 53**  
- **R² ≈ 0.49**

This means the model explains **approximately 49% of the variance** in the disease-progression target variable. In other words, it captures about half of the underlying signal — which is **typical performance** for this dataset when using linear models.

Linear methods are intentionally simple and interpretable, and may not fully capture complex, nonlinear medical relationships. However, they offer a **strong and transparent baseline**, which is especially valuable in healthcare contexts.

### Interpretation of coefficients

The learned coefficients show how each feature influences the prediction, assuming the others stay constant:

- Positive coefficients (e.g., `bmi`, `S5`) **increase predicted disease progression**
- Negative coefficients (e.g., `bp`, `S3`) **decrease predicted progression**

Examples:

- **BMI (~25.21)** has a strong positive effect — consistent with medical expectations.
- **S3 (~-34.26)** has the largest negative effect, indicating an inverse relationship.
- **Sex (~-11.44)** shows a moderate influence — note this reflects correlation in this dataset, not causation.

Overall, coefficients reveal **multiple competing physiological factors**, which is expected in biomedical data.

### Takeaways

- Ridge regression provides **reasonable accuracy** and **clear interpretability**
- Performance matches known benchmarks for this dataset
- For improved performance, future steps could include:
  - Polynomial or interaction features
  - Tree-based models (Random Forest, Gradient Boosting)
  - Non-linear models with regularization
  - Neural networks (with tuning)

Linear regression remains a valuable **first step and transparent baseline model** in medical ML workflows.


## When Linear Regression Fails (Non-linear Data)

Linear regression is a powerful tool, but it has a fundamental limitation:  
it **assumes that the relationship between \(X\) and \(y\) is linear (a straight line)**.

If the true relationship is more complex (e.g., sinusoidal, parabolic, exponential), a linear model will fail — no matter how long we train it or how much data we provide.  
We’ll demonstrate this using data that follow a sine function.



In [ ]:
# When linear regression fails (non-linear data)

# 1. Create strongly non-linear data (sine wave + noise)
rng = np.random.default_rng(RSTATE)
X_nl = np.linspace(-5, 5, 200).reshape(-1, 1)
y_nl = np.sin(X_nl).ravel() + rng.normal(scale=0.3, size=X_nl.shape[0])

# 2. Train a simple linear model
# No scaler or complex pipeline needed here
lin_reg_fail = LinearRegression()
lin_reg_fail.fit(X_nl, y_nl)

# 3. Predictions
y_pred_fail = lin_reg_fail.predict(X_nl)

# 4. Evaluation
r2_fail = r2_score(y_nl, y_pred_fail)
print(f"--- Performance on non-linear data ---")
print(f"R²: {r2_fail:.3f}")
print("The model failed to explain almost any variance!")

# 5. Visualization of failure
plt.figure(figsize=(12, 7))
plt.scatter(X_nl, y_nl, alpha=0.6, label="True non-linear data (sine wave)")
plt.plot(X_nl, y_pred_fail, color='red', lw=3,
         label=f"Linear regression (R² = {r2_fail:.3f})")
plt.title("Failure of linear regression on non-linear data")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
# Applying Polynomial Features

from sklearn.preprocessing import PolynomialFeatures

# 1. Create mildly non-linear data (parabola + noise)
rng = np.random.default_rng(RSTATE)
X_poly_src = np.linspace(-4, 4, 200).reshape(-1, 1)
# y = 0.5*x^2 + 1*x + 2 + noise
y_poly = 0.5 * (X_poly_src**2).ravel() + 1.0 * X_poly_src.ravel() + 2.0 + rng.normal(scale=1.5, size=X_poly_src.shape[0])

# Train–test split
Xtr_p, Xte_p, ytr_p, yte_p = train_test_split(X_poly_src, y_poly, test_size=0.3, random_state=RSTATE)

# --- Model 1: Linear baseline ---
# This model should fail
pipe_lin = Pipeline([
    ('scale', StandardScaler()),
    ('model', LinearRegression())
])
pipe_lin.fit(Xtr_p, ytr_p)
y_pred_lin = pipe_lin.predict(Xte_p)
rmse_lin = np.sqrt(mean_squared_error(yte_p, y_pred_lin))

# --- Model 2: Polynomial (degree = 2) ---
# This model should succeed
pipe_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Step 1: Create x and x^2
    ('scale', StandardScaler()),                                 # Step 2: Scale x and x^2
    ('model', LinearRegression())                                # Step 3: Fit the model
])
pipe_poly.fit(Xtr_p, ytr_p)
y_pred_poly = pipe_poly.predict(Xte_p)
rmse_poly = np.sqrt(mean_squared_error(yte_p, y_pred_poly))

# --- Comparison & visualization ---
print(f"RMSE (Linear model):     {rmse_lin:.3f}")
print(f"RMSE (Polynomial d=2):   {rmse_poly:.3f}  <- Much better!")

# Plot results on the test data
plt.figure(figsize=(12, 7))
plt.scatter(Xte_p, yte_p, alpha=0.6, label="Test data (parabola)")

# Sort points for smooth curves
sort_idx = np.argsort(Xte_p.ravel())
plt.plot(Xte_p[sort_idx], y_pred_lin[sort_idx], color='red', lw=3,
         label=f"Poor linear fit (RMSE: {rmse_lin:.3f})")
plt.plot(Xte_p[sort_idx], y_pred_poly[sort_idx], color='green', lw=3,
         label=f"Good polynomial fit (RMSE: {rmse_poly:.3f})")

plt.title("Rescuing Linear Regression with Polynomial Features")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()
